## Lexical Density and Variation Features

In [1]:
from collections import Counter, defaultdict
import de_core_news_sm
from math import sqrt, log
from nltk import word_tokenize
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("datasets/03_LexicalDiversity_df.csv", sep="|", lineterminator = '\n')

In [3]:
#Lexical Density #Tok Lex / #Tok 
#Lexical Word Variation #Typ Lex / #Tok Lex 
#Noun Variation #Typ Noun / #Tok Lex
#Adjective Variation #Typ Adj / #Tok Lex
#Adverb Variation #Typ Adv / #Tok Lex
#Modifer Variation (#Typ Adj + #Typ Adv) / #Tok Lex 
#Verb Variation 2 #Typ Verb / #Tok Verb 
#Verb Variation 1 #Typ Verb / #Tok Lex
#Squared Verb Variation 1 #Typ Verb^2 / #Tok Verb 
#Corrected Verb Variation 1 #Typ Verb / sqrt(2*#Tok Verb) 
#Verb Token Ratio #Tok Verb/#Tok
#Noun Token Ratio #Tok Noun/#Tok 
#Verb-Noun Token Ratio #Tok Verb/#Tok Noun 

In [4]:
nlp = de_core_news_sm.load()

In [5]:
lexical_tags = ["ADJA", "ADJD", "ADV", "NE", "NNE", "NN", "VVFIN", "VVIMP", "VVINF", "VVIZU", "VVPP"]

In [6]:
def lexical_density(text):
    doc = nlp(text)
    no_lex = len([tok for tok in doc if tok.tag_ in lexical_tags])
    return no_lex / len(doc)

In [7]:
def lexical_word_variation(text):
    doc = nlp(text)
    no_lex = [str(tok) for tok in doc if tok.tag_ in lexical_tags]
    return len(set(no_lex)) / len(no_lex)
    

In [8]:
def pos_variation(text, pos):
    doc = nlp(text)
    no_nouns = len([tok for tok in doc if tok.pos_ == pos])
    no_lex = len([tok for tok in doc if tok.tag_ in lexical_tags])
    return no_nouns / no_lex

In [9]:
def noun_variation(text):
    return pos_variation(text, "NOUN")

In [10]:
def adj_variation(text):
    return pos_variation(text, "ADJ")

In [11]:
def adv_variation(text):
    return pos_variation(text, "ADV")

In [12]:
def modifier_variation(text):
    doc = nlp(text)
    no_mods = len([tok for tok in doc if tok.pos_ == "ADJ"  or tok.pos_ == "ADV"])
    no_lex = len([tok for tok in doc if tok.tag_ in lexical_tags])
    return no_mods / no_lex

In [13]:
def verb_variation1(text):
    return pos_variation(text, "VERB")

In [14]:
def verb_variation2(text):
    doc = nlp(text)
    verbs = [str(tok) for tok in doc if tok.pos_ == "VERB"]
    if len(verbs) != 0:
        return len(set(verbs)) / len(verbs)
    else:
        return 0

In [15]:
def squared_verb_variation(text):
    doc = nlp(text)
    verbs = [str(tok) for tok in doc if tok.pos_ == "VERB"]
    if len(verbs) != 0:
        return (len(set(verbs)) ** 2) / len(verbs)
    else:
        return 0

In [16]:
def corrected_verb_variation(text):
    doc = nlp(text)
    verbs = [str(tok) for tok in doc if tok.pos_ == "VERB"]
    if len(verbs) != 0:
        return len(set(verbs)) / (sqrt(2 * len(verbs)))
    else:
        return 0

In [17]:
def verb_token_ratio(text):
    doc = nlp(text)
    verbs = [str(tok) for tok in doc if tok.pos_ == "VERB"]
    return len(verbs) / len(doc)

In [18]:
def noun_token_ratio(text):
    doc = nlp(text)
    nouns = [str(tok) for tok in doc if tok.pos_ == "NOUN"]
    return len(nouns) / len(doc)

In [19]:
def verb_noun_token_ratio(text):
    doc = nlp(text)
    nouns = [str(tok) for tok in doc if tok.pos_ == "NOUN"]
    verbs = [str(tok) for tok in doc if tok.pos_ == "VERB"]
    if len(nouns) != 0:
        return len(verbs) / len(nouns)
    else:
        return 0

In [20]:
LV_feature_functions = [lexical_density, lexical_word_variation, noun_variation, adj_variation, adv_variation, 
                        modifier_variation, verb_variation1, verb_variation2, squared_verb_variation, 
                        corrected_verb_variation, verb_token_ratio, noun_token_ratio, verb_noun_token_ratio]

In [21]:
for function in tqdm(LV_feature_functions):
    df["LV-"+function.__name__]  = [function(text) for text in df["cleanedText"]]

100%|██████████| 13/13 [33:35<00:00, 155.05s/it]


In [22]:
def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

filename = "datasets/04_LexicalDensityVariation_df.csv"
save_df(filename, df)

In [23]:
df.head()

,Level,Title,Text,Source,Type,newLevel,cleanedText,SE_ratioRealSpellingErrors,SE_ratioSpellingErrors,SE_ratioSpellingErrorsWithCorrection,...,LV-adj_variation,LV-adv_variation,LV-modifier_variation,LV-verb_variation1,LV-verb_variation2,LV-squared_verb_variation,LV-corrected_verb_variation,LV-verb_token_ratio,LV-noun_token_ratio,LV-verb_noun_token_ratio
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,A,Zu meiner Familie gehören vier Personen. Die M...,0.0,0.0,0.0,...,0.076923,0.250000,0.326923,0.250000,0.846154,9.307692,2.157277,0.109244,0.201681,0.541667
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,A,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,0.0,0.0,0.0,...,0.109589,0.136986,0.246575,0.164384,0.916667,10.083333,2.245366,0.069767,0.197674,0.352941
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,A,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,0.0,0.0,0.0,...,0.081633,0.102041,0.183673,0.122449,0.666667,2.666667,1.154701,0.046512,0.209302,0.222222
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,A,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,0.0,0.0,0.0,...,0.095238,0.206349,0.301587,0.238095,0.666667,6.666667,1.825742,0.130435,0.139130,0.937500
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,A,Frau Meier geht heute in den Supermarkt. Ihr M...,0.0,0.0,0.0,...,0.056338,0.112676,0.169014,0.211268,0.600000,10.800000,2.323790,0.103093,0.247423,0.416667
